In [1]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

In [2]:
from astropy.utils import iers
iers.conf.auto_download = False

In [3]:
d = salobj.Domain()

In [4]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")
ataos = salobj.Remote(d, "ATAOS")
atpne = salobj.Remote(d, "ATPneumatics")
athex = salobj.Remote(d, "ATHexapod")
atdome = salobj.Remote(d, "ATDome", index=1)
atdomtraj = salobj.Remote(d, "ATDomeTrajectory")

In [5]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task,
                     ataos.start_task,
                     atpne.start_task,
                     athex.start_task,
                     atdome.start_task,
                     atdomtraj.start_task)

[None, None, None, None, None, None, None]

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 10 messages


# ATMCS

In [17]:
await asyncio.sleep(5.)
print("Done")
await salobj.set_summary_state(atmcs, salobj.State.ENABLED)

RemoteTelemetry(ATDome, 1, position) falling behind; read 29 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 29 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 12 messages
falling behind; queue contains 11 elements


Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [15]:
await salobj.set_summary_state(atmcs, salobj.State.STANDBY)

AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 25 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 57 messages
falling behind; queue contains 24 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, torqueDemand) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, nasymth_m3_mountMotorEncoders) falling behind; read 11 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 57 messages
RemoteTelemetry(ATMCS, 0, mount_Nasmyth_Encoders) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, mount_AzEl_Encoders) falling behind; read 11 messages
RemoteTelemetry(ATMCS, 0, azEl_mountMotorEncoders) falling behind; read 11 messages


[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [8]:
await asyncio.sleep(5.)
print("Done")

await asyncio.gather(salobj.set_summary_state(atmcs, salobj.State.ENABLED),
                     salobj.set_summary_state(atptg, salobj.State.ENABLED))

RemoteTelemetry(ATDome, 1, position) falling behind; read 42 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 42 messages
falling behind; queue contains 20 elements


Done


[[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>],
 [<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]]

# ATPtg

In [16]:
await salobj.set_summary_state(atptg, salobj.State.STANDBY)

RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 18 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 17 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 18 messages


[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [18]:
await asyncio.sleep(5.)
print("Done")
await atptg.tel_timeAndDate.next(flush=True, timeout=5)
await salobj.set_summary_state(atptg, salobj.State.ENABLED)

RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 35 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 34 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 10 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 15 messages
falling behind; queue contains 14 elements


Done


[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await atptg.cmd_azElTarget.set_start(azDegs=-85, elDegs=80.)

In [9]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [10]:
ra = Angle("05:35:17.3", unit=u.hour)
dec = Angle("-05:23:28", unit=u.deg)
target_name="M42"
radec = ICRS(ra, dec)

In [19]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

RemoteTelemetry(ATDome, 1, position) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 28 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 15 messages
falling behind; queue contains 14 elements
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 28 messages
RemoteTelemetry(ATPtg, 0, guidingAndOffsets) falling behind; read 28 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 10 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 28 messages


58739.337346334505
slew...


In [12]:
await atptg.cmd_stopTracking.start(timeout=10)

RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 11 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 11 messages
RemoteEvent(ATAOS, 0, detailedState) falling behind; read 17 messages
falling behind; queue contains 24 elements


In [20]:
await atptg.cmd_offsetAzEl.set_start(el=-30., 
                                     az=30., 
                                         num=0)

RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 77 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 78 messages
AckCmdReader(ATMCS, 0, ackcmd) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 77 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 77 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 77 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 78 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 78 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 58 messages
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 77 messages
RemoteTelemetry(ATPneumatics, 0, m2AirPressure) falling behind

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-10., 
                                     az=20., 
                                         num=0)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-20., 
                                     az=40., 
                                         num=0)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await asyncio.gather(atmcs.close(), 
                     atptg.close(),
                     ataos.close(),
                     atpne.close(),
                     athex.close(),
                     atdome.close(),
                     atdomtraj.close())

In [ ]:
20*360/24., 22*360/24.

In [ ]:
await atptg.cmd_stopTracking.start(timeout=10)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.35, u=0.35, v=0.22)

In [21]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20, u=0.35, v=0.22)

RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 59 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 60 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 45 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 60 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 60 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 45 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 59 messages
RemoteEvent(ATPneumatics, 0, m1SetPressure) falling behind; read 45 messages
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
falling behind; queue contains 97 elements
falling behind; queue contains 34 elements
falling behind; queue contains 99 elements
falling behind; queue contains 98 elements
fall

In [ ]:
exptime = np.linspace(1, 20, 5)

In [ ]:
exptime

In [ ]:
for e in exptime:
    print(f'{target_name.replace(" ","_")}_{e:05.2f}s')
    await cam.cmd_takeImages.set_start(numImages=3,
                                       expTime=e,
                                       shutter=True,
                                       imageSequenceName=f'{target_name.replace(" ","_")}_{e:05.2f}s'
                                        )


In [ ]:
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=20.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_intra_20s'
                                    )

In [ ]:
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=20.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_extra_20s'
                                    )

In [ ]:
await asyncio.sleep(1.)

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20+0.5, u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=5,
                                     expTime=20.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_intra_z05_20s'
                                    )
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20-0.5, u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=5,
                                     expTime=20.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_extra_z05_20s'
                                    )

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20+0.75, u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=40.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_intra_40s'
                                    )
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20-0.75, u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=40.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_extra_40s'
                                    )

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20+1., u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=40.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_intra_40s'
                                    )
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20-1., u=0.35, v=0.22)
await cam.cmd_takeImages.set_start(numImages=1,
                                     expTime=40.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_extra_40s'
                                    )

In [22]:
cam = salobj.Remote(d, "GenericCamera", 1)

In [23]:
await cam.start_task

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, currentTargetStatus) falling behind; read 92 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 91 messages
AckCmdReader(ATPneumatics, 0, ackcmd) falling behind; read 41 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 18 messages
falling behind; queue contains 40 elements
RemoteTelemetry(ATMCS, 0, trajectory) falling behind; read 18 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 92 messages
RemoteEvent(ATDome, 1, heartbeat) falling behind; read 18 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 13 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 18 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 18 messages
RemoteEvent(ATDomeTrajectory, 0, heartbeat) falling behind; read 19 m

In [24]:
import numpy as np

In [ ]:
z_seq = np.linspace(0.1, 0.35, 6)
print(z_seq)

In [ ]:
z_seq[3:]

In [ ]:
focus_seq = 1

In [ ]:
for z in [z_seq[-1]]:
    print(f"Positioning hexapod @ z={z:4.2f}")
    await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=z, u=0.35, v=0.22)

    print(f"Taking images: {target_name}_z{z:4.2f}_10s")
    await cam.cmd_takeImages.set_start(numImages=3,
                                     expTime=10.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ", "_")}_z{z:4.2f}_10s'
                                    )


In [ ]:
3*360/24.

In [26]:
await cam.cmd_takeImages.set_start(numImages=10,
                                     expTime=1.,
                                     shutter=True,
                                     imageSequenceName=f'{target_name.replace(" ","_")}_test_10s'
                                    )

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(GenericCamera, 1, heartbeat) falling behind; read 100 messages
RemoteTelemetry(ATPneumatics, 0, loadCell) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, timeAndDate) falling behind; read 100 messages
RemoteTelemetry(ATDome, 1, position) falling behind; read 100 messages
RemoteEvent(ATHexapod, 0, heartbeat) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionStarted) falling behind; read 95 messages
RemoteTelemetry(ATPneumatics, 0, mainAirSourcePressure) falling behind; read 100 messages
RemoteTelemetry(ATHexapod, 0, positionStatus) falling behind; read 100 messages
RemoteTelemetry(ATPtg, 0, mountStatus) falling behind; read 100 messages
RemoteEvent(ATAOS, 0, m1CorrectionCompleted) falling behind; read 95 messages
RemoteTelemetry(ATPneumati

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20, u=0.35, v=0.22)

In [ ]:
await athex.cmd_moveToPosition.set_start(x=-4., y=1.,z=0.20, u=0.35, v=0.22)

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-30., az=0., num=1)

In [ ]:
# Take image

# offset -30 el

# Take image 

# offset +30 el 

# take image

# offset +30 el 

# take image

# offset -30 el

# take image

# offset -0.4 x
